In [1]:
import mlflow
import optax
import pandas as pd
import polars as pl
from flax import nnx
from nn_trainer.flax.evaluator import MeanSquaredErrorEvaluator
from nn_trainer.flax.loader import MiniBatchLoader
from nn_trainer.flax.loss_fn import mean_squared_error
from nn_trainer.flax.trainer import Trainer
from sklearn.model_selection import train_test_split

from nn_recsys.model.two_tower import TwoTower

### READ

In [2]:
rating_df, movie_df, user_df = (
    pl.from_pandas(
        pd.read_csv(
            "../../dataset/ML1M/ml-1m/ratings.dat",
            delimiter="::",
            engine="python",
            header=None,
            names=["user_id", "item_id", "rating", "timestamp"],
        )
    ),
    pl.from_pandas(
        pd.read_csv(
            "../../dataset/ML1M/ml-1m/movies.dat",
            delimiter="::",
            engine="python",
            header=None,
            names=["item_id", "title", "genres"],
            encoding="ISO-8859-1",
        )
    ).with_columns(pl.col("genres").str.split("|")),
    pl.from_pandas(
        pd.read_csv(
            "../../dataset/ML1M/ml-1m/users.dat",
            delimiter="::",
            engine="python",
            header=None,
            names=["user_id", "gender", "age", "occupation", "zip_code"],
            # encoding="ISO-8859-1",
        )
    ),
)

dataset_df = (
    rating_df.join(movie_df, how="left", on="item_id")
    .join(user_df, how="left", on="user_id")
    .select(pl.all().exclude("rating", "genres"), pl.col("rating"))
    .with_columns(pl.all().exclude("rating").rank("dense") - 1)
)

user_categorical_feature_indices = [0, 4, 5, 6, 7]
user_categorical_feature_cardinalities = [
    dataset_df[:, idx].n_unique() for idx in user_categorical_feature_indices
]
user_numerical_feature_indices = []
item_categorical_feature_indices = [1, 3]
item_categorical_feature_cardinalities = [
    dataset_df[:, idx].n_unique() for idx in item_categorical_feature_indices
]
item_numerical_feature_indices = []

train_df, valid_df = train_test_split(
    dataset_df, test_size=0.1, random_state=0, shuffle=True
)
dataset_df

user_id,item_id,timestamp,title,gender,age,occupation,zip_code,rating
u32,u32,u32,u32,u32,u32,u32,u32,i64
0,1104,397442,2452,0,0,10,1588,5
0,639,397457,1739,0,0,10,1588,3
0,853,397454,2289,0,0,10,1588,3
0,3177,397440,1054,0,0,10,1588,4
0,2162,400250,557,0,0,10,1588,5
…,…,…,…,…,…,…,…,…
6039,1019,382,3574,1,2,6,466,1
6039,1022,21,814,1,2,6,466,5
6039,548,17,3578,1,2,6,466,5


### 学習

In [3]:
model = TwoTower(
    user_categorical_feature_indices=user_categorical_feature_indices,
    user_categorical_feature_cardinalities=user_categorical_feature_cardinalities,
    user_numerical_feature_indices=user_numerical_feature_indices,
    user_hidden_layer_dims=[128, 64],
    item_categorical_feature_indices=item_categorical_feature_indices,
    item_categorical_feature_cardinalities=item_categorical_feature_cardinalities,
    item_numerical_feature_indices=item_numerical_feature_indices,
    item_hidden_layer_dims=[128, 64],
    embed_dim=30,
    output_layer_dim=30,
    rngs=nnx.Rngs(0),
)

In [4]:
train_df.drop("rating")

user_id,item_id,timestamp,title,gender,age,occupation,zip_code
u32,u32,u32,u32,u32,u32,u32,u32
5276,1539,188991,3161,1,2,20,2871
2149,2512,298992,1327,0,1,4,905
3255,3550,188967,3679,1,2,15,2919
3668,468,158774,1825,1,2,14,2029
1584,1374,299123,923,1,6,20,2938
…,…,…,…,…,…,…,…
5808,2110,13877,2286,1,2,7,1288
760,2501,335873,135,1,1,7,3438
2663,627,233429,2193,1,3,7,1749


In [5]:
loader = MiniBatchLoader(
    X_df=train_df.drop("rating"), y_df=train_df.select("rating"), batch_size=512, seed=0
)
evaluator = MeanSquaredErrorEvaluator(
    X_df=valid_df.drop("rating"), y_df=valid_df.select("rating")
)

In [6]:
mlflow.set_tracking_uri(uri="http://localhost:8080")
mlflow.set_experiment("TWO_TOWER_RATING")

with mlflow.start_run() as run:
    trainer = Trainer(
        model=model,
        optimizer=optax.adamw(learning_rate=0.001, weight_decay=0.001),
        train_loader=loader,
        loss_fn=mean_squared_error,
        valid_evaluator=evaluator,
        early_stopping_patience=10,
        epoch_num=32,
        active_run=run,
    )
    trainer = trainer.fit()

2025/12/30 12:21:27 INFO mlflow.tracking.fluent: Experiment with name 'TWO_TOWER_RATING' does not exist. Creating a new experiment.


[VALID 000]: loss=14.053701400756836, metrics={'MSE': 14.053701400756836}


[TRAIN 001]: 100%|██████████| 1758/1758 [00:17<00:00, 100.34it/s, batch_loss=0.742]


[VALID 001]: loss=0.8371427059173584, metrics={'MSE': 0.8371427059173584}


[TRAIN 002]: 100%|██████████| 1758/1758 [00:14<00:00, 119.68it/s, batch_loss=0.8]  


...

[VALID 016]: loss=0.7829759120941162, metrics={'MSE': 0.7829759120941162}


[TRAIN 017]: 100%|██████████| 1758/1758 [00:17<00:00, 100.10it/s, batch_loss=0.561]


[VALID 017]: loss=0.7865433692932129, metrics={'MSE': 0.7865433692932129}


[TRAIN 018]: 100%|██████████| 1758/1758 [00:17<00:00, 100.31it/s, batch_loss=0.594]


[VALID 018]: loss=0.7886098027229309, metrics={'MSE': 0.7886098027229309}
🏃 View run zealous-pug-794 at: http://localhost:8080/#/experiments/720292585535908405/runs/dce0845089c1438c90b2cae45f3327fa
🧪 View experiment at: http://localhost:8080/#/experiments/720292585535908405
